# Building a Logistic Regression Model from scratch in Python

* Built a logistic regession classsifier from scratch on a real world dataset in python without using scikit learn.
* This covers all the preprocessing steps required on this dataset before feeding it to logistic regression model.
* How to handle categorical variables.
* How to handle class imbalance.
* Build a logisitc regression model from scratch with gradient descent.
* Finally, compare the results to scikit learn logistic regression algorithm.

In [1]:
# Load the necessary libraries 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from imblearn.over_sampling import SMOTE
import statsmodels.api as sm

C:\Users\nayan\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Set the seed to 123
random.seed(123)

In [3]:
# importing the dataset
data = pd.read_csv('Logistic_regression.csv')
print(data.shape)
print(list(data.columns))

(41188, 21)
['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y']


In [4]:
# checking for missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
age               41188 non-null int64
job               41188 non-null object
marital           41188 non-null object
education         41188 non-null object
default           41188 non-null object
housing           41188 non-null object
loan              41188 non-null object
contact           41188 non-null object
month             41188 non-null object
day_of_week       41188 non-null object
duration          41188 non-null int64
campaign          41188 non-null int64
pdays             41188 non-null int64
previous          41188 non-null int64
poutcome          41188 non-null object
emp_var_rate      41188 non-null float64
cons_price_idx    41188 non-null float64
cons_conf_idx     41188 non-null float64
euribor3m         41188 non-null float64
nr_employed       41188 non-null float64
y                 41188 non-null int64
dtypes: float64(5), int64(6), object(10)
memory usag

There are no missing values in the dataset

In [5]:
# understanding data fields
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,44,blue-collar,married,Basic,unknown,yes,no,cellular,aug,thu,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,53,technician,married,unknown,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,management,single,university.degree,no,yes,no,cellular,jun,thu,...,3,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,services,married,high.school,no,no,no,cellular,apr,fri,...,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,retired,married,Basic,no,yes,no,cellular,aug,fri,...,1,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,1


In [6]:
# checking data types
data.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp_var_rate      float64
cons_price_idx    float64
cons_conf_idx     float64
euribor3m         float64
nr_employed       float64
y                   int64
dtype: object

In [7]:
# understanding categorical variables
data["education"].unique()

array(['Basic', 'unknown', 'university.degree', 'high.school',
       'professional.course', 'illiterate'], dtype=object)

### Handling categorical variables

* column 'y' is the output variable. 
* 10 variables are categorical and need to be converted to one_hot_encoded. Below are those 10 variables:
* ['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']


In [6]:
# encoding categorical variables to one hot encoding
cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
'''

for column in cat_vars:
    cat_list='column'+'_'+column
    cat_list = pd.get_dummies(data[column], prefix=column)
    data1=data.join(cat_list)
    data=data1
cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
data_vars=data.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]

'''
one_hot_encoded_vars = data[cat_vars]
one_hot_encoded_vars = pd.get_dummies(one_hot_encoded_vars, columns= cat_vars)
one_hot_encoded_vars.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 51 columns):
job_admin.                       41188 non-null uint8
job_blue-collar                  41188 non-null uint8
job_entrepreneur                 41188 non-null uint8
job_housemaid                    41188 non-null uint8
job_management                   41188 non-null uint8
job_retired                      41188 non-null uint8
job_self-employed                41188 non-null uint8
job_services                     41188 non-null uint8
job_student                      41188 non-null uint8
job_technician                   41188 non-null uint8
job_unemployed                   41188 non-null uint8
job_unknown                      41188 non-null uint8
marital_divorced                 41188 non-null uint8
marital_married                  41188 non-null uint8
marital_single                   41188 non-null uint8
marital_unknown                  41188 non-null uint8
education_Basic            

In [7]:
# making a list of columns to keep for further analysis
merged_df = pd.concat([data, one_hot_encoded_vars], axis=1)
col_to_keep = ['age', 'campaign', 'pdays', 'previous', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired',
       'job_self-employed', 'job_services', 'job_student',
       'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_Basic', 'education_high.school',
       'education_illiterate', 'education_professional.course',
       'education_university.degree', 'education_unknown', 'default_no',
       'default_unknown', 'default_yes', 'housing_no', 'housing_unknown',
       'housing_yes', 'loan_no', 'loan_unknown', 'loan_yes',
       'contact_cellular', 'contact_telephone', 'month_apr', 'month_aug',
       'month_dec', 'month_jul', 'month_jun', 'month_mar', 'month_may',
       'month_nov', 'month_oct', 'month_sep', 'day_of_week_fri',
       'day_of_week_mon', 'day_of_week_thu', 'day_of_week_tue',
       'day_of_week_wed', 'poutcome_failure', 'poutcome_nonexistent',
       'poutcome_success']
data_final = merged_df[col_to_keep]
data_final.head()

,age,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,44,1,999,0,1.4,93.444,-36.1,4.963,5228.1,0,...,0,0,0,0,1,0,0,0,1,0
1,53,1,999,0,-0.1,93.200,-42.0,4.021,5195.8,0,...,0,0,1,0,0,0,0,0,1,0
2,28,3,6,2,-1.7,94.055,-39.8,0.729,4991.6,1,...,0,0,0,0,1,0,0,0,0,1
3,39,2,999,0,-1.8,93.075,-47.1,1.405,5099.1,0,...,0,0,1,0,0,0,0,0,1,0
4,55,1,3,1,-2.9,92.201,-31.4,0.869,5076.2,1,...,0,0,1,0,0,0,0,0,0,1


cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(data[var], prefix=var)
    data1=data.join(cat_list)
    data=data1
cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
data_vars=data.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]

In [9]:
# Checking for class imbalance/balance
data_final["y"].value_counts()

0    36548
1     4640
Name: y, dtype: int64

In [10]:
'''
separate the features and the target variable 
'''

X = data_final.loc[:, data_final.columns != 'y']
y = data_final.loc[:, data_final.columns == 'y']



In [11]:
'''
splitting the training and testing data and 
as the target class is imbalanced we need to use SMOTE function to balance it.
We first separate your data into training and testing set.
then use SMOTE function on the Training set.
Remember to not touch the testing set.
'''
from sklearn.cross_validation import train_test_split
smote = SMOTE(random_state = 123)

# splitting the data in train and test samples
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns



X_balanced , y_balanced = smote.fit_sample(X_train, y_train)
X_balanced = pd.DataFrame(data = X_balanced , columns=columns )
y_balanced = pd.DataFrame(data = y_balanced , columns=['y'])

C:\Users\nayan\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\nayan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
cols = ['euribor3m', 'job_blue-collar', 'job_housemaid', 'marital_unknown', 'education_illiterate', 'default_no', 'default_unknown', 
      'contact_cellular', 'contact_telephone', 'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun', 'month_mar', 
      'month_may', 'month_nov', 'month_oct', "poutcome_failure", "poutcome_success"]

In [13]:
X_bal_1 = X_balanced[cols]
y_bal_1 = y_balanced['y']

X_bal_1.head()

,euribor3m,job_blue-collar,job_housemaid,marital_unknown,education_illiterate,default_no,default_unknown,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,poutcome_failure,poutcome_success
0,4.153,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,4.857,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,4.857,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4.153,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.266,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [14]:
#import statsmodels.api as sm

model_lm =sm.Logit(y_bal_1,X_bal_1)

model_lm_fit=model_lm.fit()

print(model_lm_fit.summary2())

         Current function value: 0.546587
         Iterations: 35
                                Results: Logit
Model:                   Logit                No. Iterations:       35.0000   
Dependent Variable:      y                    Pseudo R-squared:     0.211     
Date:                    2019-05-03 15:55     AIC:                  55938.3481
No. Observations:        51134                BIC:                  56115.1922
Df Model:                19                   Log-Likelihood:       -27949.   
Df Residuals:            51114                LL-Null:              -35443.   
Converged:               0.0000               Scale:                1.0000    
------------------------------------------------------------------------------
                      Coef.    Std.Err.    z     P>|z|     [0.025     0.975]  
------------------------------------------------------------------------------
euribor3m             -0.4570    0.0089 -51.2426 0.0000     -0.4745    -0.4396
job_blue-collar   

C:\Users\nayan\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [15]:
# Removing four columns with p-value larger than 0.05

cols_updated =['euribor3m', 'job_blue-collar', 'job_housemaid', 'marital_unknown', 'education_illiterate', 
      'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun', 'month_mar', 
      'month_may', 'month_nov', 'month_oct', "poutcome_failure", "poutcome_success"] 


X_final = X_balanced[cols_updated]
y_final = y_balanced['y']

X_final.head()




,euribor3m,job_blue-collar,job_housemaid,marital_unknown,education_illiterate,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,poutcome_failure,poutcome_success
0,4.153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,4.857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,4.857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4.153,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [16]:
# checking the p values 

model_lm_2 = sm.Logit(y_final,X_final)

model_lm_2_fit = model_lm_2.fit()

print(model_lm_2_fit.summary2())

Optimization terminated successfully.
         Current function value: 0.558479
         Iterations 7
                           Results: Logit
Model:                Logit             No. Iterations:    7.0000    
Dependent Variable:   y                 Pseudo R-squared:  0.194     
Date:                 2019-05-03 15:55  AIC:               57146.4840
No. Observations:     51134             BIC:               57287.9592
Df Model:             15                Log-Likelihood:    -28557.   
Df Residuals:         51118             LL-Null:           -35443.   
Converged:            1.0000            Scale:             1.0000    
---------------------------------------------------------------------
                      Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
---------------------------------------------------------------------
euribor3m            -0.4555   0.0072 -63.2352 0.0000 -0.4696 -0.4414
job_blue-collar      -0.1929   0.0273  -7.0745 0.0000 -0.2463 -0.1395
job_housemaid   

In [17]:
# sigmoid function
def sigmoid(features, weights):
    '''
    parameters : scores 
    does : calculate the sigmoid value of the scores
    return : the sigmoid value
    '''
    z = np.dot(features, weights)
    return 1 / (1 + np.exp(-z))

In [18]:
# calculating log likelihood
def log_likelihood(features, target, weights):
    '''
    paramters : Input variables, Target variable and weights 
    does : calculate the log-likelihood 
    return : the log-likelihood
    '''
    m = features.shape[0]
    log_lik = (1/m) * (- target * np.log(sigmoid(features,weights)) - (1-target)*np.log(1 - sigmoid(features,weights)))
    return log_lik

In [19]:
# building the logistic regression function with gradient descent
def logistic_regression(features, target, num_steps, learning_rate, add_intercept = False):
    '''
    parameters : features, target, num_steps, learning_rate and add_intercept = False
    does : calculate the logistic regression weights, i have provided the add_intercept section of the code, to run you need to change the value from False to True 
    return : The logistic regression weights
    '''
    # Don't modify this part 
    if add_intercept:
        intercept = np.ones((features.shape[0], 1))
        features = np.hstack((intercept, features))

    # Your code goes here
    weights = np.zeros(features.shape[1])
    
    ''' need to iterate over the number of steps and update the weights in each iteration based on the gradient '''
    for step in range(num_steps):
        # Calculate the prediction value
        prediction = sigmoid(features,weights)
        gradient = np.dot(np.transpose(features), target - prediction) / features.shape[0]
        weights = weights + (learning_rate * gradient)


        # Update weights with gradient

        # Print log-likelihood every so often - don't change this part 
        if step % 10000 == 0:
            print(log_likelihood(features, target, weights))
            
    return weights

In [20]:
# training first model with learning rate = 0.3 and 20000 iterations
train_model = logistic_regression(X_final,y_final,20000, 0.3,add_intercept = True)
train_model

0        0.000009
1        0.000008
2        0.000008
3        0.000009
4        0.000012
5        0.000008
6        0.000008
7        0.000009
8        0.000008
9        0.000008
10       0.000008
11       0.000008
12       0.000008
13       0.000008
14       0.000008
15       0.000008
16       0.000012
17       0.000008
18       0.000012
19       0.000008
20       0.000012
21       0.000012
22       0.000009
23       0.000008
24       0.000008
25       0.000008
26       0.000008
27       0.000008
28       0.000012
29       0.000013
           ...   
51104    0.000021
51105    0.000015
51106    0.000021
51107    0.000015
51108    0.000015
51109    0.000021
51110    0.000015
51111    0.000015
51112    0.000014
51113    0.000021
51114    0.000015
51115    0.000015
51116    0.000014
51117    0.000014
51118    0.000014
51119    0.000019
51120    0.000015
51121    0.000015
51122    0.000015
51123    0.000015
51124    0.000015
51125    0.000021
51126    0.000015
51127    0.000014
51128    0

array([ 2.10290265, -0.49717641, -0.19694523, -0.26531038,  0.33352381,
        0.52118662, -0.66820392, -0.45436645, -0.25481672, -0.21149369,
       -0.4021144 ,  0.49655676, -1.27693272, -0.57230198,  0.44685192,
       -0.60291187,  1.57244695])

In [21]:
# training first model with learning rate = 0.1 and 10000 iterations
train_model_1 = logistic_regression(X_final,y_final,10000, 0.1, add_intercept = True)
train_model_1

0        0.000012
1        0.000012
2        0.000012
3        0.000012
4        0.000013
5        0.000012
6        0.000012
7        0.000012
8        0.000012
9        0.000012
10       0.000012
11       0.000012
12       0.000012
13       0.000012
14       0.000012
15       0.000012
16       0.000013
17       0.000012
18       0.000013
19       0.000012
20       0.000013
21       0.000013
22       0.000012
23       0.000012
24       0.000012
25       0.000012
26       0.000012
27       0.000012
28       0.000013
29       0.000013
           ...   
51104    0.000016
51105    0.000014
51106    0.000016
51107    0.000014
51108    0.000014
51109    0.000016
51110    0.000014
51111    0.000014
51112    0.000014
51113    0.000016
51114    0.000014
51115    0.000014
51116    0.000014
51117    0.000014
51118    0.000014
51119    0.000015
51120    0.000014
51121    0.000014
51122    0.000014
51123    0.000014
51124    0.000014
51125    0.000016
51126    0.000014
51127    0.000014
51128    0

array([ 1.81790718, -0.50132394, -0.20016766, -0.25529841,  0.11223925,
        0.11295587, -0.37177315, -0.14799696,  0.07998493,  0.0966204 ,
       -0.09616559,  0.80161884, -0.97884569, -0.26249356,  0.76435501,
       -0.59944028,  1.58371426])

In [22]:
# weights for the the train_model 1 - these are the weights for the variables in logistic regression model.
train_model_1

array([ 1.81790718, -0.50132394, -0.20016766, -0.25529841,  0.11223925,
        0.11295587, -0.37177315, -0.14799696,  0.07998493,  0.0966204 ,
       -0.09616559,  0.80161884, -0.97884569, -0.26249356,  0.76435501,
       -0.59944028,  1.58371426])

In [23]:
# prediction function to predict outcomes for test set
def predict(test_features, model):
    return sigmoid(test_features,model)

In [24]:
# adding column for theta0 in test set

X_test = X_test[cols_updated]
inter = np.ones((X_test.shape[0], 1))
X_test = np.hstack((inter, X_test))

#test_pred = predict(X_test[cols_updated],train_model_1)

In [26]:
type(X_test) , X_test.shape

(numpy.ndarray, (12357, 17))

In [27]:
# predicting values for test set
test_pred = predict(X_test,train_model_1)

In [28]:
# checking the dimensions of predictions on test set
test_pred.shape

(12357,)

In [34]:
y_test = y_test.values

In [46]:
y_test = y_test.flatten()

In [47]:
# checking the type and shape of test target variable
type(y_test) , y_test.shape

(numpy.ndarray, (12357,))

In [50]:
# creating a function for accuracy measure to evaluate model performance
def accuracy(predicted_values, actual_values, decision_threshold=0.5):
    predicted_classes = (predicted_values >= 
                         decision_threshold).astype(int)
    predicted_classes = predicted_classes.flatten()
    accuracy = np.mean(predicted_classes == actual_values)
    return predicted_classes , round(accuracy * 100 , 2)

In [51]:
# accuracy of model
pred_classes , model_accuracy = accuracy(test_pred, y_test)

In [52]:
model_accuracy

78.27

#### The accurcacy of our model is 78.27% ,i.e, it correctly predicted classes for 78% of records in the test set. 

In [57]:
X_test.shape

(12357, 17)

In [59]:
# removing the additional theta0 column as it is not needed while inputting data in sklearn.
X_test_sk = X_test[: , 1:]

In [62]:
X_test_sk.shape

(12357, 16)

In [63]:
# Implemting the scikit-learn logistic regression classifier.
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 


classifier = LogisticRegression()
classifier.fit(X_final, y_final)
predicted_classes = classifier.predict(X_test_sk)
accuracy_skmodel = accuracy_score(y_test,predicted_classes)
weights = classifier.coef_

In [66]:
round((accuracy_skmodel * 100),2)

78.6

In [67]:
weights

array([[-0.49660068, -0.19656977, -0.26373808,  0.35909777,  0.91161625,
        -0.71465385, -0.50234334, -0.32067601, -0.25975773, -0.44976226,
         0.43998764, -1.32339806, -0.6214139 ,  0.39216758, -0.60206526,
         1.56615234]])

The accuracy of the Sklearn model is just slightly higher. So our model is not that bad. But the model trained on sklearn algorithm trains much faster as it has advanced optimization techniques used and also has regularization is which used to avoid overfitting, this is the reason the weights (model parameters) in our model are sligthly different than in the sklearn model.

In [69]:
# Making the Confusion Matrix for sklearn model
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predicted_classes)
cm

array([[8776, 2205],
       [ 439,  937]], dtype=int64)

In [70]:
# Making the Confusion Matrix for our model
from sklearn.metrics import confusion_matrix
cm_model = confusion_matrix(y_test, pred_classes)
cm_model

array([[8732, 2249],
       [ 436,  940]], dtype=int64)

## Conclusion

As the intention of this exercise is to understand implementation/working of the logistic regression model, I have not done a full blown accuracy analysis. This exercise is just for learning purpose.